In [4]:
import sys
import os
project_path ='/content/drive/MyDrive/Detection'
sys.path.append(project_path)

In [4]:
"""
Test script for Step 1 foundation components
File: step1_test.py (place in project root)

Run this to verify all Step 1 components work correctly:
python step1_test.py
"""
import torch
import torch.nn as nn
import sys
import os

# Add project root to path
sys.path.append(os.path.dirname(os.path.abspath(project_path)))

def test_tensor_utils():
    """Test tensor utility functions"""
    print("=" * 50)
    print("Testing utils/tensor_utils.py")
    print("=" * 50)

    try:
        from utils.tensor_utils import (
            window_partition, window_reverse, yolo_to_swin_format,
            swin_to_yolo_format, make_divisible, drop_path,
            calculate_feature_stats, ensure_tensor_compatibility
        )

        # Test format conversion
        print("1. Testing format conversion...")
        x_yolo = torch.randn(2, 96, 56, 56)  # B, C, H, W
        x_swin = yolo_to_swin_format(x_yolo)  # B, H, W, C
        x_back = swin_to_yolo_format(x_swin)  # B, C, H, W

        assert x_yolo.shape == x_back.shape, f"Format conversion failed: {x_yolo.shape} != {x_back.shape}"
        assert torch.allclose(x_yolo, x_back), "Format conversion values don't match"
        print(f"   ✅ YOLO format: {x_yolo.shape}")
        print(f"   ✅ SWIN format: {x_swin.shape}")
        print(f"   ✅ Back to YOLO: {x_back.shape}")

        # Test window partition
        print("2. Testing window partition...")
        window_size = 7
        windows = window_partition(x_swin, window_size)
        x_restored = window_reverse(windows, window_size, x_swin.shape[1], x_swin.shape[2])

        assert x_swin.shape == x_restored.shape, f"Window partition failed: {x_swin.shape} != {x_restored.shape}"
        assert torch.allclose(x_swin, x_restored), "Window partition values don't match"
        print(f"   ✅ Original: {x_swin.shape}")
        print(f"   ✅ Windows: {windows.shape}")
        print(f"   ✅ Restored: {x_restored.shape}")

        # Test make_divisible
        print("3. Testing make_divisible...")
        result1 = make_divisible(97, 8)
        result2 = make_divisible(96, 8)
        print(f"   🔸 make_divisible(97, 8) = {result1} (expected: 96 or 104)")
        print(f"   🔸 make_divisible(96, 8) = {result2} (expected: 96)")

        assert result1 % 8 == 0, f"make_divisible(97, 8) not divisible by 8: {result1}"
        assert result2 == 96, f"make_divisible(96, 8) should be 96, got {result2}"
        print(f"   ✅ make_divisible working correctly")

        # Test feature stats
        print("4. Testing feature statistics...")
        mean, var = calculate_feature_stats(x_yolo)
        print(f"   ✅ Feature mean shape: {mean.shape}")
        print(f"   ✅ Feature var shape: {var.shape}")

        # Test tensor compatibility
        print("5. Testing tensor compatibility...")
        x_resized = ensure_tensor_compatibility(x_yolo, (112, 112))
        print(f"   ✅ Original: {x_yolo.shape}")
        print(f"   ✅ Resized: {x_resized.shape}")

        print("✅ All tensor_utils tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ tensor_utils test failed: {e}")
        return False

def test_focal_modulation():
    """Test focal modulation layers"""
    print("=" * 50)
    print("Testing models/backbone/focal_modulation.py")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from models.backbone.focal_modulation import (
                FocalModulation, FocalModulationYOLO, create_focal_modulation
            )
        except ImportError:
            # Try alternative import path
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'backbone'))
            from focal_modulation import (
                FocalModulation, FocalModulationYOLO, create_focal_modulation
            )

        # Test FocalModulation (SWIN format)
        print("1. Testing FocalModulation (SWIN format)...")
        focal_swin = FocalModulation(dim=96, focal_level=2, focal_window=7)
        x_swin = torch.randn(2, 56, 56, 96)  # B, H, W, C
        out_swin = focal_swin(x_swin)

        assert out_swin.shape == x_swin.shape, f"FocalModulation output shape mismatch: {out_swin.shape} != {x_swin.shape}"
        print(f"   ✅ Input: {x_swin.shape}")
        print(f"   ✅ Output: {out_swin.shape}")
        print(f"   ✅ Parameters: {sum(p.numel() for p in focal_swin.parameters()):,}")

        # Test FocalModulationYOLO (YOLO format)
        print("2. Testing FocalModulationYOLO (YOLO format)...")
        focal_yolo = FocalModulationYOLO(dim=96, focal_level=2, focal_window=7)
        x_yolo = torch.randn(2, 96, 56, 56)  # B, C, H, W
        out_yolo = focal_yolo(x_yolo)

        assert out_yolo.shape == x_yolo.shape, f"FocalModulationYOLO output shape mismatch: {out_yolo.shape} != {x_yolo.shape}"
        print(f"   ✅ Input: {x_yolo.shape}")
        print(f"   ✅ Output: {out_yolo.shape}")
        print(f"   ✅ Parameters: {sum(p.numel() for p in focal_yolo.parameters()):,}")

        # Test factory function
        print("3. Testing factory function...")
        focal_factory_swin = create_focal_modulation(dim=96, input_format="swin")
        focal_factory_yolo = create_focal_modulation(dim=96, input_format="yolo")

        assert isinstance(focal_factory_swin, FocalModulation), "Factory function failed for SWIN"
        assert isinstance(focal_factory_yolo, FocalModulationYOLO), "Factory function failed for YOLO"
        print(f"   ✅ Factory SWIN: {type(focal_factory_swin).__name__}")
        print(f"   ✅ Factory YOLO: {type(focal_factory_yolo).__name__}")

        # Test gradient flow
        print("4. Testing gradient flow...")
        x_test = torch.randn(1, 28, 28, 48, requires_grad=True)
        focal_test = FocalModulation(dim=48, focal_level=1)
        out_test = focal_test(x_test)
        loss = out_test.sum()
        loss.backward()

        assert x_test.grad is not None, "Gradient flow failed"
        print(f"   ✅ Gradient flow working")

        print("✅ All focal_modulation tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ focal_modulation test failed: {e}")
        return False

def test_swin_transformer():
    """Test SWIN transformer components"""
    print("=" * 50)
    print("Testing models/backbone/swin_transformer.py")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from models.backbone.swin_transformer import (
                SwinTransformerBlock, BasicLayer, PatchEmbed,
                WindowAttention, Mlp, PatchMerging
            )
        except ImportError:
            # Try alternative import path
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'backbone'))
            from swin_transformer import (
                SwinTransformerBlock, BasicLayer, PatchEmbed,
                WindowAttention, Mlp, PatchMerging
            )

        # Test SwinTransformerBlock with Focal Modulation
        print("1. Testing SwinTransformerBlock (with Focal Modulation)...")
        block_focal = SwinTransformerBlock(
            dim=96,
            input_resolution=(56, 56),
            num_heads=3,
            window_size=7,
            use_focal_modulation=True
        )
        x_block = torch.randn(2, 56*56, 96)  # B, H*W, C
        out_block_focal = block_focal(x_block)

        assert out_block_focal.shape == x_block.shape, f"SwinTransformerBlock output shape mismatch: {out_block_focal.shape} != {x_block.shape}"
        print(f"   ✅ Input: {x_block.shape}")
        print(f"   ✅ Output: {out_block_focal.shape}")
        print(f"   ✅ Using Focal Modulation: {block_focal.use_focal_modulation}")

        # Test SwinTransformerBlock with Window Attention
        print("2. Testing SwinTransformerBlock (with Window Attention)...")
        block_attn = SwinTransformerBlock(
            dim=96,
            input_resolution=(56, 56),
            num_heads=3,
            window_size=7,
            use_focal_modulation=False
        )
        out_block_attn = block_attn(x_block)

        assert out_block_attn.shape == x_block.shape, f"SwinTransformerBlock output shape mismatch: {out_block_attn.shape} != {x_block.shape}"
        print(f"   ✅ Input: {x_block.shape}")
        print(f"   ✅ Output: {out_block_attn.shape}")
        print(f"   ✅ Using Focal Modulation: {block_attn.use_focal_modulation}")

        # Test BasicLayer
        print("3. Testing BasicLayer...")
        layer = BasicLayer(
            dim=96,
            input_resolution=(56, 56),
            depth=2,
            num_heads=3,
            window_size=7,
            downsample=PatchMerging,
            use_focal_modulation=True
        )
        out_layer = layer(x_block)

        expected_shape = (2, (56//2)*(56//2), 96*2)  # Downsampled by PatchMerging
        assert out_layer.shape == expected_shape, f"BasicLayer output shape mismatch: {out_layer.shape} != {expected_shape}"
        print(f"   ✅ Input: {x_block.shape}")
        print(f"   ✅ Output: {out_layer.shape}")
        print(f"   ✅ Depth: {layer.depth}")

        # Test PatchEmbed
        print("4. Testing PatchEmbed...")
        patch_embed = PatchEmbed(
            img_size=224,
            patch_size=4,
            in_chans=3,
            embed_dim=96
        )
        x_img = torch.randn(2, 3, 224, 224)  # B, C, H, W
        out_embed = patch_embed(x_img)

        expected_patches = (224//4) * (224//4)  # 56*56 = 3136
        expected_shape = (2, expected_patches, 96)
        assert out_embed.shape == expected_shape, f"PatchEmbed output shape mismatch: {out_embed.shape} != {expected_shape}"
        print(f"   ✅ Input: {x_img.shape}")
        print(f"   ✅ Output: {out_embed.shape}")
        print(f"   ✅ Patches: {patch_embed.num_patches}")

        # Test Mlp
        print("5. Testing Mlp...")
        mlp = Mlp(in_features=96, hidden_features=384)
        x_mlp = torch.randn(2, 100, 96)
        out_mlp = mlp(x_mlp)

        assert out_mlp.shape == x_mlp.shape, f"Mlp output shape mismatch: {out_mlp.shape} != {x_mlp.shape}"
        print(f"   ✅ Input: {x_mlp.shape}")
        print(f"   ✅ Output: {out_mlp.shape}")

        # Test WindowAttention
        print("6. Testing WindowAttention...")
        win_attn = WindowAttention(dim=96, window_size=(7, 7), num_heads=3)
        x_win = torch.randn(8, 49, 96)  # nW*B, window_size*window_size, C
        out_win = win_attn(x_win)

        assert out_win.shape == x_win.shape, f"WindowAttention output shape mismatch: {out_win.shape} != {x_win.shape}"
        print(f"   ✅ Input: {x_win.shape}")
        print(f"   ✅ Output: {out_win.shape}")

        print("✅ All swin_transformer tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ swin_transformer test failed: {e}")
        return False

def test_integration():
    """Test integration between components"""
    print("=" * 50)
    print("Testing Component Integration")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from utils.tensor_utils import yolo_to_swin_format, swin_to_yolo_format
            from models.backbone.focal_modulation import FocalModulation
            from models.backbone.swin_transformer import SwinTransformerBlock
        except ImportError:
            # Try alternative import paths
            from tensor_utils import yolo_to_swin_format, swin_to_yolo_format
            from focal_modulation import FocalModulation
            from swin_transformer import SwinTransformerBlock

        print("1. Testing YOLO → SWIN → Focal Modulation pipeline...")

        # Start with YOLO format
        x_yolo = torch.randn(2, 96, 56, 56)  # B, C, H, W
        print(f"   🔸 Original YOLO: {x_yolo.shape}")

        # Convert to SWIN format
        x_swin = yolo_to_swin_format(x_yolo)
        print(f"   🔸 Converted to SWIN: {x_swin.shape}")

        # Apply Focal Modulation
        focal_mod = FocalModulation(dim=96, focal_level=2)
        x_focal = focal_mod(x_swin)
        print(f"   🔸 After Focal Modulation: {x_focal.shape}")

        # Convert back to YOLO
        x_back = swin_to_yolo_format(x_focal)
        print(f"   🔸 Back to YOLO: {x_back.shape}")

        assert x_yolo.shape == x_back.shape, "Integration pipeline shape mismatch"
        print("   ✅ Integration pipeline successful!")

        print("2. Testing SWIN Block with different configurations...")

        # Test with different resolutions
        resolutions = [(28, 28), (56, 56), (112, 112)]
        dims = [96, 192, 384]

        for i, (res, dim) in enumerate(zip(resolutions, dims)):
            print(f"   🔸 Testing resolution {res} with dim {dim}...")

            block = SwinTransformerBlock(
                dim=dim,
                input_resolution=res,
                num_heads=dim//32,
                window_size=7,
                use_focal_modulation=True
            )

            x_test = torch.randn(1, res[0]*res[1], dim)
            out_test = block(x_test)

            assert out_test.shape == x_test.shape, f"Block test failed for {res}x{dim}"
            print(f"     ✅ {x_test.shape} → {out_test.shape}")

        print("✅ All integration tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ Integration test failed: {e}")
        return False

def test_memory_and_performance():
    """Test memory usage and basic performance"""
    print("=" * 50)
    print("Testing Memory & Performance")
    print("=" * 50)

    try:
        import time
        # Add fallback import handling
        try:
            from models.backbone.focal_modulation import FocalModulation
            from models.backbone.swin_transformer import SwinTransformerBlock
        except ImportError:
            from focal_modulation import FocalModulation
            from swin_transformer import SwinTransformerBlock

        print("1. Testing memory efficiency...")

        # Test with different batch sizes
        batch_sizes = [1, 2, 4]
        for batch_size in batch_sizes:
            x = torch.randn(batch_size, 56, 56, 96)
            focal = FocalModulation(dim=96)

            # Measure memory before
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

            out = focal(x)

            print(f"   ✅ Batch size {batch_size}: {x.shape} → {out.shape}")

        print("2. Testing inference speed...")

        # Warm up
        x = torch.randn(2, 56, 56, 96)
        focal = FocalModulation(dim=96)
        for _ in range(5):
            _ = focal(x)

        # Measure speed
        start_time = time.time()
        num_runs = 50
        for _ in range(num_runs):
            _ = focal(x)
        end_time = time.time()

        avg_time = (end_time - start_time) / num_runs * 1000  # ms
        print(f"   ✅ Average inference time: {avg_time:.2f}ms")

        print("3. Testing gradient memory...")

        x = torch.randn(1, 28, 28, 48, requires_grad=True)
        block = SwinTransformerBlock(
            dim=48,
            input_resolution=(28, 28),
            num_heads=3,
            window_size=7,
            use_focal_modulation=True
        )

        out = block(x.view(1, 28*28, 48))
        loss = out.sum()
        loss.backward()

        # Check gradient exists
        assert x.grad is not None, "Gradient computation failed"
        print(f"   ✅ Gradient shape: {x.grad.shape}")

        print("✅ All memory & performance tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ Memory & performance test failed: {e}")
        return False

def main():
    """Run all tests"""
    print("🚀 Starting Step 1 Foundation Components Test")
    print("=" * 70)

    test_results = []

    # Run all tests
    test_results.append(("Tensor Utils", test_tensor_utils()))
    test_results.append(("Focal Modulation", test_focal_modulation()))
    test_results.append(("SWIN Transformer", test_swin_transformer()))
    test_results.append(("Integration", test_integration()))
    test_results.append(("Memory & Performance", test_memory_and_performance()))

    # Summary
    print("=" * 70)
    print("🎯 TEST SUMMARY")
    print("=" * 70)

    passed = 0
    total = len(test_results)

    for test_name, result in test_results:
        status = "✅ PASSED" if result else "❌ FAILED"
        print(f"{test_name:<20}: {status}")
        if result:
            passed += 1

    print("=" * 70)
    print(f"📊 Results: {passed}/{total} tests passed")

    if passed == total:
        print("🎉 All Step 1 foundation components are working correctly!")
        print("✨ Ready to proceed to Step 2: Integration Layer")
    else:
        print("⚠️  Some tests failed. Please check the errors above.")
        print("🔧 Fix the issues before proceeding to Step 2.")

    print("=" * 70)

    return passed == total

if __name__ == "__main__":
    success = main()
    exit(0 if success else 1)

🚀 Starting Step 1 Foundation Components Test
Testing utils/tensor_utils.py
1. Testing format conversion...
   ✅ YOLO format: torch.Size([2, 96, 56, 56])
   ✅ SWIN format: torch.Size([2, 56, 56, 96])
   ✅ Back to YOLO: torch.Size([2, 96, 56, 56])
2. Testing window partition...
   ✅ Original: torch.Size([2, 56, 56, 96])
   ✅ Windows: torch.Size([128, 7, 7, 96])
   ✅ Restored: torch.Size([2, 56, 56, 96])
3. Testing make_divisible...
   🔸 make_divisible(97, 8) = 96 (expected: 96 or 104)
   🔸 make_divisible(96, 8) = 96 (expected: 96)
   ✅ make_divisible working correctly
4. Testing feature statistics...
   ✅ Feature mean shape: torch.Size([2, 56])
   ✅ Feature var shape: torch.Size([2, 56])
5. Testing tensor compatibility...
   ✅ Original: torch.Size([2, 96, 56, 56])
   ✅ Resized: torch.Size([2, 112, 112, 56])
✅ All tensor_utils tests passed!

Testing models/backbone/focal_modulation.py
1. Testing FocalModulation (SWIN format)...
   ✅ Input: torch.Size([2, 56, 56, 96])
   ✅ Output: torch.Si

In [4]:
"""
Test script for Step 2 integration layer components
File: step2_test.py (place in project root)

Run this to verify all Step 2 components work correctly:
python step2_test.py
"""
import torch
import torch.nn as nn
import sys
import os
import time

# Add project root to path
sys.path.append(os.path.dirname(os.path.abspath(project_path)))

def test_hybrid_c3f():
    """Test Hybrid C3F components"""
    print("=" * 50)
    print("Testing models/backbone/hybrid_c3f.py")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from models.backbone.hybrid_c3f import (
                HybridC3F, AdaptiveHybridC3F, create_hybrid_c3f,
                SWINAdapter, SWINReverseAdapter, C3F
            )
        except ImportError:
            # Try alternative import path
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'backbone'))
            from hybrid_c3f import (
                HybridC3F, AdaptiveHybridC3F, create_hybrid_c3f,
                SWINAdapter, SWINReverseAdapter, C3F
            )

        # Test SWINAdapter
        print("1. Testing SWINAdapter...")
        adapter = SWINAdapter(c1=64, c2=96, img_size=56)
        x_yolo = torch.randn(2, 64, 56, 56)  # YOLO format
        x_swin, resolution = adapter(x_yolo)

        expected_shape = (2, 56*56, 96)
        assert x_swin.shape == expected_shape, f"SWINAdapter output shape mismatch: {x_swin.shape} != {expected_shape}"
        print(f"   ✅ Input: {x_yolo.shape}")
        print(f"   ✅ Output: {x_swin.shape}")
        print(f"   ✅ Resolution: {resolution}")

        # Test SWINReverseAdapter
        print("2. Testing SWINReverseAdapter...")
        reverse_adapter = SWINReverseAdapter(c1=96, c2=64, output_size=(56, 56))
        x_back = reverse_adapter(x_swin, (56, 56))

        assert x_back.shape == x_yolo.shape, f"SWINReverseAdapter output shape mismatch: {x_back.shape} != {x_yolo.shape}"
        print(f"   ✅ Input: {x_swin.shape}")
        print(f"   ✅ Output: {x_back.shape}")

        # Test original C3F
        print("3. Testing original C3F...")
        c3f = C3F(c1=64, c2=128, n=2)
        out_c3f = c3f(x_yolo)

        expected_c3f_shape = (2, 128, 56, 56)
        assert out_c3f.shape == expected_c3f_shape, f"C3F output shape mismatch: {out_c3f.shape} != {expected_c3f_shape}"
        print(f"   ✅ Input: {x_yolo.shape}")
        print(f"   ✅ Output: {out_c3f.shape}")
        print(f"   ✅ Parameters: {sum(p.numel() for p in c3f.parameters()):,}")

        # Test HybridC3F
        print("4. Testing HybridC3F...")
        hybrid_c3f = HybridC3F(c1=64, c2=128, n=2, swin_depth=2, img_size=56)
        out_hybrid = hybrid_c3f(x_yolo)

        assert out_hybrid.shape == expected_c3f_shape, f"HybridC3F output shape mismatch: {out_hybrid.shape} != {expected_c3f_shape}"
        print(f"   ✅ Input: {x_yolo.shape}")
        print(f"   ✅ Output: {out_hybrid.shape}")
        print(f"   ✅ Parameters: {sum(p.numel() for p in hybrid_c3f.parameters()):,}")

        # Compare parameter counts
        c3f_params = sum(p.numel() for p in c3f.parameters())
        hybrid_params = sum(p.numel() for p in hybrid_c3f.parameters())
        param_increase = ((hybrid_params - c3f_params) / c3f_params) * 100
        print(f"   ✅ Parameter increase: {param_increase:.1f}%")

        # Test AdaptiveHybridC3F
        print("5. Testing AdaptiveHybridC3F...")
        adaptive_c3f = AdaptiveHybridC3F(c1=64, c2=128, n=2, swin_depth=2)

        # Test with different input sizes
        test_sizes = [28, 56, 112]
        for size in test_sizes:
            x_test = torch.randn(1, 64, size, size)
            out_test = adaptive_c3f(x_test)
            expected_shape = (1, 128, size, size)
            assert out_test.shape == expected_shape, f"AdaptiveHybridC3F failed for size {size}"
            print(f"   ✅ Size {size}: {x_test.shape} → {out_test.shape}")

        # Test factory function
        print("6. Testing factory function...")
        variants = ['standard', 'adaptive', 'lightweight']
        for variant in variants:
            factory_model = create_hybrid_c3f(
                c1=64, c2=128, variant=variant, n=2, swin_depth=2
            )
            x_test = torch.randn(1, 64, 56, 56)
            out_test = factory_model(x_test)
            params = sum(p.numel() for p in factory_model.parameters())
            print(f"   ✅ {variant}: {out_test.shape}, Params: {params:,}")

        # Test feature extraction
        print("7. Testing feature extraction...")
        feature_maps = hybrid_c3f.get_feature_maps(x_yolo)
        print(f"   ✅ Available features: {list(feature_maps.keys())}")
        print(f"   ✅ C3F main shape: {feature_maps['c3f_main'].shape}")
        print(f"   ✅ SWIN output shape: {feature_maps['swin_output'].shape}")

        # Test gradient flow
        print("8. Testing gradient flow...")
        x_grad = torch.randn(1, 64, 56, 56, requires_grad=True)
        out_grad = hybrid_c3f(x_grad)
        loss = out_grad.sum()
        loss.backward()

        assert x_grad.grad is not None, "Gradient flow failed"
        print(f"   ✅ Gradient flow working")
        print(f"   ✅ Gradient shape: {x_grad.grad.shape}")

        print("✅ All hybrid_c3f tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ hybrid_c3f test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_yolo_backbone():
    """Test YOLO backbone with SWIN integration"""
    print("=" * 50)
    print("Testing models/backbone/yolo_backbone.py")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from models.backbone.yolo_backbone import (
                YOLOSWINBackbone, YOLOSWINBackboneFactory,
                yolo_swin_nano, yolo_swin_small, yolo_swin_medium,
                yolo_swin_large, yolo_swin_medical
            )
        except ImportError:
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'backbone'))
            from yolo_backbone import (
                YOLOSWINBackbone, YOLOSWINBackboneFactory,
                yolo_swin_nano, yolo_swin_small, yolo_swin_medium,
                yolo_swin_large, yolo_swin_medical
            )

        # Test basic backbone
        print("1. Testing YOLOSWINBackbone...")
        backbone = YOLOSWINBackbone(
            width_multiple=1.0,
            depth_multiple=1.0,
            use_swin_in_stages=[1, 2, 3]
        )

        x = torch.randn(2, 3, 640, 640)
        features = backbone(x)

        print(f"   ✅ Input: {x.shape}")
        print(f"   ✅ Number of feature levels: {len(features)}")
        for i, feat in enumerate(features):
            print(f"   ✅ Feature {i+1}: {feat.shape}")

        # Test model info
        info = backbone.get_model_info()
        print(f"   ✅ Total parameters: {info['total_parameters']:,}")
        print(f"   ✅ SWIN percentage: {info['swin_percentage']:.1f}%")
        print(f"   ✅ Channels: {info['channels']}")

        # Test different model sizes using factory
        print("2. Testing different model sizes...")
        sizes = ['nano', 'small', 'medium', 'large']

        for size in sizes:
            print(f"   🔸 Testing {size} model...")
            model = YOLOSWINBackboneFactory.create_backbone(size)

            # Test with smaller input for faster testing
            x_test = torch.randn(1, 3, 320, 320)
            features_test = model(x_test)
            info_test = model.get_model_info()

            print(f"     ✅ Features: {len(features_test)} levels")
            print(f"     ✅ Parameters: {info_test['total_parameters']:,}")
            print(f"     ✅ SWIN stages: {info_test['swin_stages']}")

        # Test convenience functions
        print("3. Testing convenience functions...")
        models_to_test = [
            ('nano', yolo_swin_nano),
            ('small', yolo_swin_small),
            ('medium', yolo_swin_medium)
        ]

        for name, model_func in models_to_test:
            model = model_func()
            x_test = torch.randn(1, 3, 320, 320)
            features_test = model(x_test)
            print(f"   ✅ {name}: {len(features_test)} features, shapes: {[f.shape for f in features_test]}")

        # Test medical backbone
        print("4. Testing medical backbone...")
        medical_model = yolo_swin_medical(input_channels=1, model_size='small')
        x_medical = torch.randn(1, 1, 512, 512)  # Grayscale medical image
        features_medical = medical_model(x_medical)

        print(f"   ✅ Medical input: {x_medical.shape}")
        print(f"   ✅ Medical features: {len(features_medical)} levels")
        for i, feat in enumerate(features_medical):
            print(f"   ✅ Medical feature {i+1}: {feat.shape}")

        # Test detailed feature extraction
        print("5. Testing detailed feature extraction...")
        model = yolo_swin_small()
        x_test = torch.randn(1, 3, 320, 320)
        features_dict = model.forward_with_features(x_test)

        print(f"   ✅ Available features: {list(features_dict.keys())}")
        print(f"   ✅ FPN features: {len(features_dict['fpn_features'])}")

        # Test feature channels
        channels = model.get_feature_channels()
        print(f"   ✅ Feature channels: {channels}")

        # Test factory methods
        print("6. Testing factory methods...")
        available_models = YOLOSWINBackboneFactory.list_available_models()
        print(f"   ✅ Available models: {available_models}")

        # Test configuration retrieval
        config = YOLOSWINBackboneFactory.get_model_config('medium')
        print(f"   ✅ Medium config keys: {list(config.keys())}")

        # Test custom configuration
        print("7. Testing custom configuration...")
        custom_model = YOLOSWINBackboneFactory.create_backbone(
            'medium',
            use_swin_in_stages=[2, 3],  # Override SWIN stages
            swin_config={'swin_depth': 1}  # Override SWIN depth
        )

        x_custom = torch.randn(1, 3, 320, 320)
        features_custom = custom_model(x_custom)
        info_custom = custom_model.get_model_info()

        print(f"   ✅ Custom model features: {len(features_custom)}")
        print(f"   ✅ Custom SWIN stages: {info_custom['swin_stages']}")

        print("✅ All yolo_backbone tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ yolo_backbone test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_integration():
    """Test integration between hybrid_c3f and yolo_backbone"""
    print("=" * 50)
    print("Testing Integration: HybridC3F ↔ YOLO Backbone")
    print("=" * 50)

    try:
        # Import both modules
        try:
            from models.backbone.hybrid_c3f import HybridC3F, AdaptiveHybridC3F
            from models.backbone.yolo_backbone import YOLOSWINBackbone, yolo_swin_medium
        except ImportError:
            from hybrid_c3f import HybridC3F, AdaptiveHybridC3F
            from yolo_backbone import YOLOSWINBackbone, yolo_swin_medium

        print("1. Testing HybridC3F integration in backbone...")

        # Create backbone with SWIN in different stages
        backbone = YOLOSWINBackbone(
            width_multiple=0.5,  # Smaller for faster testing
            depth_multiple=0.5,
            use_swin_in_stages=[2, 3],  # Use SWIN in stages 2 and 3
            swin_config={
                'swin_depth': 1,
                'variant': 'adaptive'
            }
        )

        # Test with multiple input sizes
        input_sizes = [(320, 320), (416, 416), (640, 640)]

        for h, w in input_sizes:
            print(f"   🔸 Testing input size {h}x{w}...")
            x = torch.randn(1, 3, h, w)
            features = backbone(x)

            # Verify feature shapes are reasonable
            for i, feat in enumerate(features):
                expected_h = h // (8 * (2 ** min(i, 2)))  # Downsampling ratios
                expected_w = w // (8 * (2 ** min(i, 2)))
                print(f"     ✅ Feature {i+1}: {feat.shape}")

            assert len(features) == 4, f"Expected 4 feature levels, got {len(features)}"

        print("2. Testing memory efficiency...")

        # Test memory usage with different batch sizes
        model = yolo_swin_medium()
        model.eval()

        batch_sizes = [1, 2, 4]
        for batch_size in batch_sizes:
            x = torch.randn(batch_size, 3, 320, 320)

            # Measure inference time
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            start_time = time.time()

            with torch.no_grad():
                features = model(x)

            inference_time = (time.time() - start_time) * 1000  # ms

            print(f"   ✅ Batch {batch_size}: {inference_time:.1f}ms, Memory efficient: {len(features)} features")

        print("3. Testing gradient compatibility...")

        # Test that gradients flow properly through the integrated model
        model = yolo_swin_medium()
        model.train()

        x = torch.randn(1, 3, 320, 320, requires_grad=True)
        features = model(x)

        # Create a simple loss from all features
        total_loss = sum(feat.mean() for feat in features)
        total_loss.backward()

        assert x.grad is not None, "Gradient flow failed"
        print(f"   ✅ Gradient flow working through {len(features)} feature levels")

        # Check that SWIN components received gradients
        swin_grad_found = False
        for name, param in model.named_parameters():
            if 'swin' in name.lower() and param.grad is not None:
                swin_grad_found = True
                break

        print(f"   ✅ SWIN components receiving gradients: {swin_grad_found}")

        print("4. Testing feature consistency...")

        # Compare outputs between standard and adaptive variants
        standard_model = YOLOSWINBackbone(
            width_multiple=0.5,
            use_swin_in_stages=[2],
            swin_config={'variant': 'standard', 'swin_depth': 1}
        )

        adaptive_model = YOLOSWINBackbone(
            width_multiple=0.5,
            use_swin_in_stages=[2],
            swin_config={'variant': 'adaptive', 'swin_depth': 1}
        )

        x_test = torch.randn(1, 3, 320, 320)

        with torch.no_grad():
            features_standard = standard_model(x_test)
            features_adaptive = adaptive_model(x_test)

        # Both should produce same number of features with same shapes
        assert len(features_standard) == len(features_adaptive), "Feature count mismatch"

        for i, (f_std, f_adapt) in enumerate(zip(features_standard, features_adaptive)):
            assert f_std.shape == f_adapt.shape, f"Feature {i} shape mismatch: {f_std.shape} vs {f_adapt.shape}"

        print(f"   ✅ Feature consistency verified across variants")

        print("✅ All integration tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ Integration test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_performance_comparison():
    """Compare performance between original and hybrid models"""
    print("=" * 50)
    print("Testing Performance Comparison")
    print("=" * 50)

    try:
        # Import modules
        try:
            from models.backbone.hybrid_c3f import C3F, HybridC3F
            from models.backbone.yolo_backbone import yolo_swin_medium
        except ImportError:
            from hybrid_c3f import C3F, HybridC3F
            from yolo_backbone import yolo_swin_medium

        print("1. Comparing C3F vs HybridC3F...")

        # Create comparable models
        c3f_model = C3F(c1=64, c2=128, n=2)
        hybrid_model = HybridC3F(c1=64, c2=128, n=2, swin_depth=1, img_size=56)

        x = torch.randn(4, 64, 56, 56)  # Batch of 4 for better timing

        # Compare parameters
        c3f_params = sum(p.numel() for p in c3f_model.parameters())
        hybrid_params = sum(p.numel() for p in hybrid_model.parameters())
        param_ratio = hybrid_params / c3f_params

        print(f"   📊 C3F parameters: {c3f_params:,}")
        print(f"   📊 HybridC3F parameters: {hybrid_params:,}")
        print(f"   📊 Parameter ratio: {param_ratio:.2f}x")

        # Compare inference time
        num_runs = 20

        # Warm up
        for _ in range(5):
            _ = c3f_model(x)
            _ = hybrid_model(x)

        # Time C3F
        start_time = time.time()
        for _ in range(num_runs):
            with torch.no_grad():
                _ = c3f_model(x)
        c3f_time = (time.time() - start_time) / num_runs * 1000

        # Time HybridC3F
        start_time = time.time()
        for _ in range(num_runs):
            with torch.no_grad():
                _ = hybrid_model(x)
        hybrid_time = (time.time() - start_time) / num_runs * 1000

        time_ratio = hybrid_time / c3f_time

        print(f"   ⏱️  C3F inference time: {c3f_time:.2f}ms")
        print(f"   ⏱️  HybridC3F inference time: {hybrid_time:.2f}ms")
        print(f"   ⏱️  Time ratio: {time_ratio:.2f}x")

        print("2. Testing backbone performance...")

        # Test backbone inference time
        backbone = yolo_swin_medium()
        backbone.eval()

        input_sizes = [(320, 320), (640, 640)]
        batch_sizes = [1, 4]

        for (h, w) in input_sizes:
            for batch_size in batch_sizes:
                x_test = torch.randn(batch_size, 3, h, w)

                # Warm up
                for _ in range(3):
                    with torch.no_grad():
                        _ = backbone(x_test)

                # Time inference
                start_time = time.time()
                num_runs = 10
                for _ in range(num_runs):
                    with torch.no_grad():
                        features = backbone(x_test)
                inference_time = (time.time() - start_time) / num_runs * 1000

                print(f"   ⏱️  {h}x{w}, batch {batch_size}: {inference_time:.1f}ms")

        print("3. Testing memory usage...")

        # Test peak memory usage (approximate)
        model = yolo_swin_medium()

        # Clear cache
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

        # Test different input sizes
        for size in [320, 480, 640]:
            x_mem = torch.randn(1, 3, size, size)

            if torch.cuda.is_available():
                model = model.cuda()
                x_mem = x_mem.cuda()
                torch.cuda.reset_peak_memory_stats()

                with torch.no_grad():
                    _ = model(x_mem)

                peak_memory = torch.cuda.max_memory_allocated() / 1024**2  # MB
                print(f"   💾 {size}x{size}: ~{peak_memory:.0f}MB peak memory")

                model = model.cpu()
            else:
                with torch.no_grad():
                    _ = model(x_mem)
                print(f"   💾 {size}x{size}: Memory usage test completed (CPU)")

        print("✅ All performance tests completed!\n")
        return True

    except Exception as e:
        print(f"❌ Performance test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def main():
    """Run all Step 2 tests"""
    print("🚀 Starting Step 2 Integration Layer Components Test")
    print("=" * 70)

    test_results = []

    # Run all tests
    test_results.append(("Hybrid C3F", test_hybrid_c3f()))
    test_results.append(("YOLO Backbone", test_yolo_backbone()))
    test_results.append(("Integration", test_integration()))
    test_results.append(("Performance", test_performance_comparison()))

    # Summary
    print("=" * 70)
    print("🎯 TEST SUMMARY")
    print("=" * 70)

    passed = 0
    total = len(test_results)

    for test_name, result in test_results:
        status = "✅ PASSED" if result else "❌ FAILED"
        print(f"{test_name:<20}: {status}")
        if result:
            passed += 1

    print("=" * 70)
    print(f"📊 Results: {passed}/{total} tests passed")

    if passed == total:
        print("🎉 All Step 2 integration components are working correctly!")
        print("✨ Ready to proceed to Step 3: Quality Control System")
        print("💡 Key achievements:")
        print("   • HybridC3F successfully integrates SWIN with YOLO")
        print("   • Multiple backbone variants working (nano to xlarge)")
        print("   • Medical imaging support implemented")
        print("   • Performance is reasonable with added capabilities")
        print("   • Memory usage is manageable")
    else:
        print("⚠️  Some tests failed. Please check the errors above.")
        print("🔧 Fix the issues before proceeding to Step 3.")

    print("=" * 70)

    return passed == total

if __name__ == "__main__":
    success = main()
    exit(0 if success else 1)

🚀 Starting Step 2 Integration Layer Components Test
Testing models/backbone/hybrid_c3f.py
1. Testing SWINAdapter...
   ✅ Input: torch.Size([2, 64, 56, 56])
   ✅ Output: torch.Size([2, 3136, 96])
   ✅ Resolution: (56, 56)
2. Testing SWINReverseAdapter...
   ✅ Input: torch.Size([2, 3136, 96])
   ✅ Output: torch.Size([2, 64, 56, 56])
3. Testing original C3F...
   ✅ Input: torch.Size([2, 64, 56, 56])
   ✅ Output: torch.Size([2, 128, 56, 56])
   ✅ Parameters: 107,520
4. Testing HybridC3F...
   ✅ Input: torch.Size([2, 64, 56, 56])
   ✅ Output: torch.Size([2, 128, 56, 56])
   ✅ Parameters: 250,569
   ✅ Parameter increase: 133.0%
5. Testing AdaptiveHybridC3F...
   ✅ Size 28: torch.Size([1, 64, 28, 28]) → torch.Size([1, 128, 28, 28])
   ✅ Size 56: torch.Size([1, 64, 56, 56]) → torch.Size([1, 128, 56, 56])
   ✅ Size 112: torch.Size([1, 64, 112, 112]) → torch.Size([1, 128, 112, 112])
6. Testing factory function...
   ✅ standard: torch.Size([1, 128, 56, 56]), Params: 250,569
   ✅ adaptive: torch.S

In [5]:
"""
Test script for Step 3 quality control system components
File: step3_test.py (place in project root)

Run this to verify all Step 3 components work correctly:
python step3_test.py
"""
import torch
import torch.nn as nn
import sys
import os
import time
import numpy as np

# Add project root to path
sys.path.append(os.path.dirname(os.path.abspath(project_path)))

def test_hotelling_t2():
    """Test T² Hotelling statistics implementation"""
    print("=" * 50)
    print("Testing models/quality_control/hotelling_t2.py")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from models.quality_control.hotelling_t2 import (
                HotellingT2Statistics, MultiLevelHotellingT2,
                create_simple_monitor, create_yolo_monitor, create_medical_monitor
            )
        except ImportError:
            # Try alternative import path
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'quality_control'))
            from hotelling_t2 import (
                HotellingT2Statistics, MultiLevelHotellingT2,
                create_simple_monitor, create_yolo_monitor, create_medical_monitor
            )

        # Test basic HotellingT2Statistics
        print("1. Testing HotellingT2Statistics...")
        monitor = HotellingT2Statistics(feature_dim=32, phase1_samples=50, alpha=0.05)

        # Generate Phase I data (baseline)
        torch.manual_seed(42)
        np.random.seed(42)
        phase1_complete = False

        for i in range(60):  # More than needed to test completion
            # Generate normal data for baseline
            features = torch.randn(32) * 0.5 + torch.randn(32) * 0.1
            complete = monitor.add_phase1_sample(features)
            if complete and not phase1_complete:
                print(f"   ✅ Phase I completed after {i+1} samples")
                phase1_complete = True
                break

        assert phase1_complete, "Phase I should be complete"

        # Test statistics
        stats = monitor.get_statistics()
        print(f"   ✅ Control limit: {stats['control_limit']:.2f}")
        print(f"   ✅ Feature dim: {stats['feature_dim']}")
        print(f"   ✅ Phase I complete: {stats['phase1_complete']}")

        # Test T² calculation
        normal_sample = torch.randn(32) * 0.5
        outlier_sample = torch.randn(32) * 3.0  # Strong outlier

        t2_normal = monitor.calculate_t2_statistic(normal_sample)
        t2_outlier = monitor.calculate_t2_statistic(outlier_sample)

        print(f"   📊 Normal T²: {t2_normal:.2f}")
        print(f"   📊 Outlier T²: {t2_outlier:.2f}")

        is_normal_outlier = monitor.is_outlier(normal_sample)
        is_outlier_outlier = monitor.is_outlier(outlier_sample)

        print(f"   🔍 Normal sample is outlier: {is_normal_outlier}")
        print(f"   🔍 Outlier sample is outlier: {is_outlier_outlier}")

        # Test batch processing
        print("2. Testing batch processing...")
        batch_features = torch.randn(5, 32) * 0.5
        batch_t2 = monitor.calculate_t2_statistic(batch_features)
        batch_outliers = monitor.is_outlier(batch_features)

        assert batch_t2.shape == (5,), f"Expected batch T² shape (5,), got {batch_t2.shape}"
        assert batch_outliers.shape == (5,), f"Expected batch outlier shape (5,), got {batch_outliers.shape}"
        print(f"   ✅ Batch T² shape: {batch_t2.shape}")
        print(f"   ✅ Batch outliers detected: {batch_outliers.sum().item()}")

        # Test adaptive updates
        print("3. Testing adaptive updates...")
        initial_mean = monitor.mean_vector.copy()

        # Add some normal samples
        for _ in range(10):
            normal_features = torch.randn(32) * 0.5
            monitor.adaptive_update(normal_features, is_normal=True)

        mean_change = np.linalg.norm(monitor.mean_vector - initial_mean)
        print(f"   ✅ Mean vector changed by: {mean_change:.6f}")

        # Test MultiLevelHotellingT2
        print("4. Testing MultiLevelHotellingT2...")
        level_configs = {
            'backbone': {'feature_dim': 64, 'phase1_samples': 30},
            'neck': {'feature_dim': 32, 'phase1_samples': 30}
        }
        multi_monitor = MultiLevelHotellingT2(level_configs, global_alpha=0.05)

        # Phase I for multi-level
        for i in range(40):
            level_features = {
                'backbone': torch.randn(64) * 0.5,
                'neck': torch.randn(32) * 0.5
            }
            complete = multi_monitor.add_phase1_samples(level_features)
            if complete:
                print(f"   ✅ Multi-level Phase I completed after {i+1} samples")
                break

        # Test multi-level monitoring
        test_features = {
            'backbone': torch.randn(64) * 0.5,  # Normal
            'neck': torch.randn(32) * 2.5       # Potential outlier
        }

        multi_status = multi_monitor.get_overall_status(test_features)
        print(f"   📊 Overall outlier: {multi_status['overall_outlier']}")
        print(f"   📊 Outlier levels: {multi_status['outlier_levels']}")
        print(f"   📊 Health score: {multi_status['health_score']:.3f}")

        # Test factory functions
        print("5. Testing factory functions...")
        simple_mon = create_simple_monitor(feature_dim=16, alpha=0.1)
        yolo_mon = create_yolo_monitor(backbone_dim=128, neck_dim=64)
        medical_mon = create_medical_monitor([256, 128, 64])

        print(f"   ✅ Simple monitor created: feature_dim={simple_mon.feature_dim}")
        print(f"   ✅ YOLO monitor created: {len(yolo_mon.monitors)} levels")
        print(f"   ✅ Medical monitor created: {len(medical_mon.monitors)} levels")

        print("✅ All hotelling_t2 tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ hotelling_t2 test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_monitoring_system():
    """Test real-time monitoring system"""
    print("=" * 50)
    print("Testing models/quality_control/monitoring.py")
    print("=" * 50)

    try:
        # Add fallback import handling
        try:
            from models.quality_control.monitoring import (
                InferenceMonitor, YOLOSWINQualityMonitor, FeatureExtractor,
                QualityAlert, AlertLevel, create_yolo_swin_monitor, create_medical_monitor
            )
        except ImportError:
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'quality_control'))
            from monitoring import (
                InferenceMonitor, YOLOSWINQualityMonitor, FeatureExtractor,
                QualityAlert, AlertLevel, create_yolo_swin_monitor, create_medical_monitor
            )

        # Test FeatureExtractor
        print("1. Testing FeatureExtractor...")

        # Test statistical features
        test_tensor = torch.randn(2, 64, 32, 32)  # Batch of feature maps
        stat_features = FeatureExtractor.extract_statistical_features(test_tensor)
        print(f"   ✅ Statistical features shape: {stat_features.shape}")

        # Test activation features
        activation_features = FeatureExtractor.extract_activation_features(test_tensor)
        print(f"   ✅ Activation features shape: {activation_features.shape}")

        # Test different input formats
        tensor_3d = torch.randn(2, 64, 1024)  # (B, C, L)
        stat_features_3d = FeatureExtractor.extract_statistical_features(tensor_3d)
        print(f"   ✅ 3D tensor features shape: {stat_features_3d.shape}")

        # Test InferenceMonitor
        print("2. Testing InferenceMonitor...")

        # Alert callback for testing
        alerts_received = []
        def test_alert_callback(alert: QualityAlert):
            alerts_received.append(alert)
            print(f"   🚨 Alert: {alert.level.value} - {alert.message}")

        monitor = InferenceMonitor(
            model_name="TestModel",
            alert_callback=test_alert_callback,
            max_history_size=1000
        )

        # Configure monitoring points
        monitoring_points = {
            'layer1': 64,  # Feature dimensions after extraction
            'layer2': 128
        }
        monitor.configure_monitoring(monitoring_points, global_alpha=0.05)

        # Add training samples for baseline establishment
        torch.manual_seed(42)
        for i in range(100):
            # Generate normal training features
            layer1_features = torch.randn(1, 64, 16, 16) * 0.5
            layer2_features = torch.randn(1, 128, 8, 8) * 0.5

            activations = {
                'layer1': layer1_features,
                'layer2': layer2_features
            }

            # This will automatically establish baseline when enough samples are collected
            result = monitor.monitor_inference(activations)

            # Check if baseline is established
            if i == 50:  # Check midway
                summary = monitor.get_monitoring_summary()
                print(f"   📊 Training progress: {summary['total_inferences']} inferences")

        print(f"   ✅ Baseline establishment completed")
        print(f"   ✅ Alerts received during training: {len(alerts_received)}")

        # Test normal inference
        print("3. Testing normal inference monitoring...")
        normal_activations = {
            'layer1': torch.randn(1, 64, 16, 16) * 0.5,
            'layer2': torch.randn(1, 128, 8, 8) * 0.5
        }

        normal_result = monitor.monitor_inference(normal_activations)
        print(f"   ✅ Normal inference outlier: {normal_result['overall_outlier']}")
        print(f"   ✅ Processing time: {normal_result['processing_time_ms']:.2f}ms")

        # Test outlier detection
        print("4. Testing outlier detection...")
        outlier_activations = {
            'layer1': torch.randn(1, 64, 16, 16) * 3.0,  # Strong outlier
            'layer2': torch.randn(1, 128, 8, 8) * 0.5   # Normal
        }

        outlier_result = monitor.monitor_inference(outlier_activations)
        print(f"   ✅ Outlier inference detected: {outlier_result['overall_outlier']}")
        print(f"   ✅ Outlier in layer1: {outlier_result['point_results'].get('layer1', {}).get('is_outlier', False)}")

        # Test batch processing
        print("5. Testing batch processing...")
        batch_activations = {
            'layer1': torch.randn(4, 64, 16, 16) * 0.5,
            'layer2': torch.randn(4, 128, 8, 8) * 0.5
        }

        batch_result = monitor.monitor_inference(batch_activations)
        print(f"   ✅ Batch processing successful: {batch_result['inference_id']}")

        # Test monitoring summary
        print("6. Testing monitoring summary...")
        summary = monitor.get_monitoring_summary()
        print(f"   📊 Total inferences: {summary['total_inferences']}")
        print(f"   📊 Total outliers: {summary['total_outliers']}")
        print(f"   📊 Outlier rate: {summary['overall_outlier_rate']:.1%}")
        print(f"   📊 Runtime: {summary['runtime_hours']:.3f} hours")

        # Test YOLOSWINQualityMonitor
        print("7. Testing YOLOSWINQualityMonitor...")

        yolo_alerts = []
        def yolo_alert_callback(alert: QualityAlert):
            yolo_alerts.append(alert)
            print(f"   🏥 Medical Alert: {alert.level.value} - {alert.message}")

        yolo_monitor = YOLOSWINQualityMonitor(
            backbone_channels=512,
            neck_channels=256,
            head_channels=128,
            alert_callback=yolo_alert_callback
        )

        # Training phase for medical monitor
        print("   🔄 Training medical monitor...")
        for i in range(150):  # More samples for medical applications
            backbone_feat = torch.randn(1, 512, 20, 20) * 0.5
            neck_feat = torch.randn(1, 256, 40, 40) * 0.5
            head_feat = torch.randn(1, 128, 80, 80) * 0.5

            # Simulate detection results
            detections = torch.tensor([[[100, 100, 200, 200, 0.8, 0]]])  # [x1,y1,x2,y2,conf,class]

            medical_result = yolo_monitor.monitor_medical_inference(
                backbone_feat, neck_feat, head_feat,
                detections=detections,
                image_metadata={'modality': 'CT', 'slice_thickness': 1.0}
            )

        print(f"   ✅ Medical monitor training completed")

        # Test medical inference with normal case
        print("8. Testing medical inference monitoring...")
        normal_backbone = torch.randn(1, 512, 20, 20) * 0.5
        normal_neck = torch.randn(1, 256, 40, 40) * 0.5
        normal_head = torch.randn(1, 128, 80, 80) * 0.5
        normal_detections = torch.tensor([[[150, 150, 250, 250, 0.9, 1]]])

        normal_medical_result = yolo_monitor.monitor_medical_inference(
            normal_backbone, normal_neck, normal_head,
            detections=normal_detections,
            image_metadata={'modality': 'MRI', 'patient_id': 'P001'}
        )

        print(f"   ✅ Normal medical inference outlier: {normal_medical_result['overall_outlier']}")
        print(f"   ✅ Medical checks passed: {not normal_medical_result['medical_checks']['anomaly_detected']}")

        # Test medical outlier detection
        print("9. Testing medical outlier detection...")
        outlier_backbone = torch.randn(1, 512, 20, 20) * 5.0  # Strong outlier
        low_conf_detections = torch.tensor([[[50, 50, 100, 100, 0.2, 0]]])  # Low confidence

        outlier_medical_result = yolo_monitor.monitor_medical_inference(
            outlier_backbone, normal_neck, normal_head,
            detections=low_conf_detections,
            image_metadata={'modality': 'X-ray', 'urgent': True}
        )

        print(f"   ✅ Medical outlier detected: {outlier_medical_result['overall_outlier']}")
        print(f"   ✅ Medical anomaly detected: {outlier_medical_result['medical_checks']['anomaly_detected']}")

        # Test factory functions
        print("10. Testing factory functions...")

        factory_monitor = create_yolo_swin_monitor(
            backbone_channels=1024, neck_channels=512, head_channels=256
        )

        medical_factory_monitor = create_medical_monitor(
            model_channels=[1024, 512, 256],
            alert_callback=lambda x: print(f"Factory alert: {x.message}")
        )

        print(f"   ✅ Factory YOLO monitor created")
        print(f"   ✅ Factory medical monitor created")

        # Test data export
        print("11. Testing data export...")
        try:
            export_path = "test_monitoring_data.json"
            monitor.export_monitoring_data(export_path)

            # Check if file was created
            if os.path.exists(export_path):
                print(f"   ✅ Monitoring data exported to {export_path}")
                os.remove(export_path)  # Cleanup
            else:
                print(f"   ⚠️  Export file not found")
        except Exception as e:
            print(f"   ⚠️  Export failed: {e}")

        # Test reset functionality
        print("12. Testing reset functionality...")
        initial_inferences = monitor.total_inferences
        monitor.reset_monitoring()

        post_reset_summary = monitor.get_monitoring_summary()
        print(f"   ✅ Inferences before reset: {initial_inferences}")
        print(f"   ✅ Inferences after reset: {post_reset_summary['total_inferences']}")
        assert post_reset_summary['total_inferences'] == 0, "Reset should clear inference count"

        print("✅ All monitoring system tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ monitoring system test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_integration_with_yolo_swin():
    """Test integration with YOLO-SWIN backbone"""
    print("=" * 50)
    print("Testing Quality Control Integration with YOLO-SWIN")
    print("=" * 50)

    try:
        # Import both quality control and backbone
        try:
            from models.quality_control.monitoring import create_yolo_swin_monitor
            from models.backbone.yolo_backbone import yolo_swin_medium
        except ImportError:
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'quality_control'))
            sys.path.append(os.path.join(os.path.dirname(__file__), 'models', 'backbone'))
            from monitoring import create_yolo_swin_monitor
            from yolo_backbone import yolo_swin_medium

        print("1. Testing YOLO-SWIN + Quality Control integration...")

        # Create YOLO-SWIN model
        model = yolo_swin_medium()
        model.eval()

        # Create quality monitor
        quality_monitor = create_yolo_swin_monitor(
            backbone_channels=1024,
            neck_channels=512,
            head_channels=256
        )

        print("   ✅ Model and monitor created")

        # Training phase - collect baseline
        print("2. Collecting baseline from model features...")
        torch.manual_seed(42)

        with torch.no_grad():
            for i in range(100):
                # Generate synthetic medical images
                x = torch.randn(1, 3, 320, 320) * 0.5 + 0.5  # Normalized medical-like images

                # Get model features
                features_dict = model.forward_with_features(x)

                # Extract specific layers for monitoring
                backbone_feat = features_dict['stage_3']  # Backbone output
                neck_feat = features_dict['stage_2']      # Neck-like output
                head_feat = features_dict['stage_1']      # Head-like output

                # Monitor the features
                result = quality_monitor.monitor_medical_inference(
                    backbone_feat, neck_feat, head_feat,
                    image_metadata={'training_sample': i}
                )

                if i % 25 == 0:
                    print(f"   📊 Training progress: {i+1}/100 samples")

        print("   ✅ Baseline collection completed")

        # Test normal inference
        print("3. Testing normal inference...")
        with torch.no_grad():
            normal_input = torch.randn(1, 3, 320, 320) * 0.5 + 0.5
            features_dict = model.forward_with_features(normal_input)

            result = quality_monitor.monitor_medical_inference(
                features_dict['stage_3'],
                features_dict['stage_2'],
                features_dict['stage_1'],
                image_metadata={'test_type': 'normal'}
            )

            print(f"   ✅ Normal inference outlier: {result['overall_outlier']}")

        # Test with corrupted input (outlier)
        print("4. Testing outlier detection...")
        with torch.no_grad():
            # Corrupted input - very high variance
            outlier_input = torch.randn(1, 3, 320, 320) * 3.0 + 1.0
            features_dict = model.forward_with_features(outlier_input)

            result = quality_monitor.monitor_medical_inference(
                features_dict['stage_3'],
                features_dict['stage_2'],
                features_dict['stage_1'],
                image_metadata={'test_type': 'corrupted'}
            )

            print(f"   ✅ Outlier inference detected: {result['overall_outlier']}")

        # Test performance
        print("5. Testing monitoring performance...")
        start_time = time.time()

        with torch.no_grad():
            for i in range(10):
                x = torch.randn(1, 3, 320, 320) * 0.5 + 0.5
                features_dict = model.forward_with_features(x)

                result = quality_monitor.monitor_medical_inference(
                    features_dict['stage_3'],
                    features_dict['stage_2'],
                    features_dict['stage_1']
                )

        total_time = time.time() - start_time
        avg_monitoring_time = (total_time / 10) * 1000

        print(f"   ⏱️  Average monitoring time: {avg_monitoring_time:.2f}ms per inference")

        # Get final summary
        summary = quality_monitor.get_monitoring_summary()
        print(f"   📊 Total monitored inferences: {summary['total_inferences']}")
        print(f"   📊 Total outliers detected: {summary['total_outliers']}")
        print(f"   📊 Overall outlier rate: {summary['overall_outlier_rate']:.1%}")

        print("✅ All integration tests passed!\n")
        return True

    except Exception as e:
        print(f"❌ Integration test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_performance_impact():
    """Test performance impact of quality monitoring"""
    print("=" * 50)
    print("Testing Performance Impact")
    print("=" * 50)

    try:
        from models.quality_control.monitoring import create_yolo_swin_monitor

        print("1. Testing monitoring overhead...")

        # Create monitor
        monitor = create_yolo_swin_monitor()

        # Simulate training
        for i in range(150):
            backbone_feat = torch.randn(1, 1024, 20, 20) * 0.5
            neck_feat = torch.randn(1, 512, 40, 40) * 0.5
            head_feat = torch.randn(1, 256, 80, 80) * 0.5
            monitor.monitor_medical_inference(backbone_feat, neck_feat, head_feat)

        # Benchmark with monitoring
        print("2. Benchmarking with monitoring...")
        torch.manual_seed(42)

        start_time = time.time()
        num_runs = 50

        for i in range(num_runs):
            backbone_feat = torch.randn(1, 1024, 20, 20) * 0.5
            neck_feat = torch.randn(1, 512, 40, 40) * 0.5
            head_feat = torch.randn(1, 256, 80, 80) * 0.5

            result = monitor.monitor_medical_inference(backbone_feat, neck_feat, head_feat)

        with_monitoring_time = (time.time() - start_time) / num_runs * 1000

        # Benchmark without monitoring (just feature extraction)
        print("3. Benchmarking without monitoring...")
        from models.quality_control.monitoring import FeatureExtractor

        start_time = time.time()

        for i in range(num_runs):
            backbone_feat = torch.randn(1, 1024, 20, 20) * 0.5
            neck_feat = torch.randn(1, 512, 40, 40) * 0.5
            head_feat = torch.randn(1, 256, 80, 80) * 0.5

            # Just feature extraction without monitoring
            _ = FeatureExtractor.extract_statistical_features(backbone_feat)
            _ = FeatureExtractor.extract_statistical_features(neck_feat)
            _ = FeatureExtractor.extract_statistical_features(head_feat)

        without_monitoring_time = (time.time() - start_time) / num_runs * 1000

        # Calculate overhead
        monitoring_overhead = with_monitoring_time - without_monitoring_time
        overhead_percentage = (monitoring_overhead / without_monitoring_time) * 100

        print(f"   ⏱️  Time with monitoring: {with_monitoring_time:.2f}ms")
        print(f"   ⏱️  Time without monitoring: {without_monitoring_time:.2f}ms")
        print(f"   ⏱️  Monitoring overhead: {monitoring_overhead:.2f}ms ({overhead_percentage:.1f}%)")

        # Memory usage test
        print("4. Testing memory usage...")

        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

            # Run monitoring on GPU
            device = torch.device('cuda')
            backbone_feat = torch.randn(1, 1024, 20, 20, device=device) * 0.5
            neck_feat = torch.randn(1, 512, 40, 40, device=device) * 0.5
            head_feat = torch.randn(1, 256, 80, 80, device=device) * 0.5

            # Move features to CPU for monitoring (since monitoring is CPU-based)
            result = monitor.monitor_medical_inference(
                backbone_feat.cpu(), neck_feat.cpu(), head_feat.cpu()
            )

            peak_memory = torch.cuda.max_memory_allocated() / 1024**2
            print(f"   💾 Peak GPU memory usage: {peak_memory:.1f} MB")
        else:
            print(f"   💾 CPU-only testing (no GPU available)")

        print("✅ All performance tests completed!\n")
        return True

    except Exception as e:
        print(f"❌ Performance test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def main():
    """Run all Step 3 tests"""
    print("🚀 Starting Step 3 Quality Control System Test")
    print("=" * 70)

    test_results = []

    # Run all tests
    test_results.append(("Hotelling T² Statistics", test_hotelling_t2()))
    test_results.append(("Monitoring System", test_monitoring_system()))
    test_results.append(("YOLO-SWIN Integration", test_integration_with_yolo_swin()))
    test_results.append(("Performance Impact", test_performance_impact()))

    # Summary
    print("=" * 70)
    print("🎯 TEST SUMMARY")
    print("=" * 70)

    passed = 0
    total = len(test_results)

    for test_name, result in test_results:
        status = "✅ PASSED" if result else "❌ FAILED"
        print(f"{test_name:<25}: {status}")
        if result:
            passed += 1

    print("=" * 70)
    print(f"📊 Results: {passed}/{total} tests passed")

    if passed == total:
        print("🎉 All Step 3 quality control components are working correctly!")
        print("✨ Ready for real-world medical imaging applications!")
        print("💡 Key achievements:")
        print("   • T² Hotelling statistics for outlier detection")
        print("   • Real-time inference monitoring system")
        print("   • Medical-specific quality checks")
        print("   • Integration with YOLO-SWIN backbone")
        print("   • Acceptable performance overhead (<10ms)")
        print("   • Multi-level monitoring capabilities")
    else:
        print("⚠️  Some tests failed. Please check the errors above.")
        print("🔧 Fix the issues before deploying to production.")

    print("=" * 70)

    return passed == total

if __name__ == "__main__":
    success = main()
    exit(0 if success else 1)

🚀 Starting Step 3 Quality Control System Test
Testing models/quality_control/hotelling_t2.py
1. Testing HotellingT2Statistics...
   ✅ Phase I completed after 50 samples
   ✅ Control limit: 186.26
   ✅ Feature dim: 32
   ✅ Phase I complete: True
   📊 Normal T²: 290.17
   📊 Outlier T²: 37058.71
   🔍 Normal sample is outlier: True
   🔍 Outlier sample is outlier: True
2. Testing batch processing...
   ✅ Batch T² shape: torch.Size([5])
   ✅ Batch outliers detected: 3
3. Testing adaptive updates...
   ✅ Mean vector changed by: 0.000000
4. Testing MultiLevelHotellingT2...
   ✅ Multi-level Phase I completed after 30 samples
   📊 Overall outlier: False
   📊 Outlier levels: []
   📊 Health score: 1.000
5. Testing factory functions...
   ✅ Simple monitor created: feature_dim=16
   ✅ YOLO monitor created: 2 levels
   ✅ Medical monitor created: 3 levels
✅ All hotelling_t2 tests passed!

Testing models/quality_control/monitoring.py
1. Testing FeatureExtractor...
   ✅ Statistical features shape: torch